In [1]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
import datetime
import time
import math

In [2]:
spark = SparkSession.builder.master("local") \
                    .appName("Homework") \
                    .config("spark.jars", "~/spark-3.3.1-bin-hadoop3/jars/postgresql-42.5.0.jar") \
                    .config("spark.driver.bindAddress", "localhost") \
                    .config("spark.ui.port", "4040") \
                    .getOrCreate()

22/11/20 20:42:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
schema = T.StructType([
                T.StructField("id", T.IntegerType(), True),
                T.StructField("timestamp", T.LongType(), True),    
                T.StructField("type", T.StringType(), True),
                T.StructField("page_id", T.IntegerType(), True),                
                T.StructField("tag", T.ArrayType(T.StringType()), True),
                T.StructField("sign", T.BooleanType(), True)
])

In [4]:
# 2000-01-01...

data = [(1, 946683757, 'visit', 1, ['политика', 'спорт'], True),
        (1, 946687357, 'click', 2, ['спорт'], True),
        (3, 946784557, 'scroll', 3, ['медицина'], True),
        (4, 946975357, 'move', 3, ['медицина'], False)
]

#columns = ["id","timestamp","type","page_id","tag","sign"]
df = spark.createDataFrame(data = data, schema = schema)

In [5]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- type: string (nullable = true)
 |-- page_id: integer (nullable = true)
 |-- tag: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- sign: boolean (nullable = true)



In [6]:
# Вывести топ-5 самых активных посетителей сайта

tmp = df.groupBy("id") \
        .agg(F.count("id").alias("cnt")) \
        .orderBy(F.col("cnt").desc())

tmp.show(5)

+---+---+
| id|cnt|
+---+---+
|  1|  2|
|  3|  1|
|  4|  1|
+---+---+



In [7]:
# Посчитать процент посетителей, у которых есть ЛК

df.filter(df.sign == True).count() / df.count() * 100

75.0

In [8]:
# Вывести топ-5 страниц сайта по показателю среднего кол-ва кликов на данной странице

tmp.groupBy("id") \
    .agg(F.avg("cnt").alias("avg")) \
    .orderBy(F.col("avg").desc()) \
    .show(5)

+---+---+
| id|avg|
+---+---+
|  1|2.0|
|  3|1.0|
|  4|1.0|
+---+---+



In [9]:
# Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

# через udf
def get_hour_window(hour):
    step = 4
    for x in range(0, 24, step):
        if hour < x:
            return f'{x-step}-{x}'
    return "20-0"

convertUDF = F.udf(lambda h: get_hour_window(h))

df = df.withColumn("hour", F.hour(F.from_unixtime(F.col("timestamp")))) \
        .withColumn("hour_window", convertUDF(F.col("hour")))

df.show()

# без udf можно типа F.when(F.col("hour").between(0, 4), "0-4").otherwise("")


+---+---------+------+-------+-----------------+-----+----+-----------+
| id|timestamp|  type|page_id|              tag| sign|hour|hour_window|
+---+---------+------+-------+-----------------+-----+----+-----------+
|  1|946683757| visit|      1|[политика, спорт]| true|   2|        0-4|
|  1|946687357| click|      2|          [спорт]| true|   3|        0-4|
|  3|946784557|scroll|      3|       [медицина]| true|   6|        4-8|
|  4|946975357|  move|      3|       [медицина]|false|  11|       8-12|
+---+---------+------+-------+-----------------+-----+----+-----------+



In [10]:
# Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.

df.groupBy('hour_window').agg(F.count('hour_window').alias("cnt")) \
    .orderBy(F.col("cnt").desc()) \
    .show(1)

+-----------+---+
|hour_window|cnt|
+-----------+---+
|        0-4|  2|
+-----------+---+
only showing top 1 row



In [11]:
schema = T.StructType([
                T.StructField("Id", T.IntegerType(), False),
                T.StructField("User_id", T.IntegerType(), False),    
                T.StructField("Full_name", T.StringType(), True),
                T.StructField("Create_date", T.DateType(), True)
])

data = [(1, 1, 'Логинов Алексей Алексеевич', datetime.datetime.strptime('1999-04-01', "%Y-%m-%d").date()),
        (2, 2, 'Калугин Максим Львович', datetime.datetime.strptime('1999-05-19', "%Y-%m-%d").date()),
        (3, 3, 'Кудрявцева Варвара Денисовна', datetime.datetime.strptime('1999-09-05', "%Y-%m-%d").date()),
        (4, 4, 'Титова Василиса Сергеевна', datetime.datetime.strptime('1999-12-01', "%Y-%m-%d").date()),
        (5, 5, 'Смирнова Екатерина Львовна', datetime.datetime.strptime('1999-01-01', "%Y-%m-%d").date())
]

df2 = spark.createDataFrame(data = data, schema = schema)

In [12]:
# Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.

(
    df
    .filter(F.array_contains(F.col("tag"), "спорт"))
    .join(df2, df.id == df2.User_id, 'inner')
    .select(df2.Full_name)
    .distinct()
    .show(truncate = False)
)

+--------------------------+
|Full_name                 |
+--------------------------+
|Логинов Алексей Алексеевич|
+--------------------------+



In [13]:
# Выведите 10% ЛК, у которых максимальная разница между датой создания ЛК и датой последнего посещения.

total = df.join(df2, df.page_id == df2.Id, 'inner').count()
 
(
    df
    .filter(F.col('type') == 'visit')
    .join(df2, df.page_id == df2.Id, 'inner')
    .withColumn('diff', F.datediff(F.from_unixtime(F.col("timestamp")), F.col("Create_date")))
    .select('diff', F.from_unixtime(F.col("timestamp")).alias('visit_date'), "Create_date")
    .orderBy(F.col("diff").desc())
    .show(math.ceil(total * 10 / 100))
)


+----+-------------------+-----------+
|diff|         visit_date|Create_date|
+----+-------------------+-----------+
| 275|2000-01-01 02:42:37| 1999-04-01|
+----+-------------------+-----------+



In [14]:
# Вывести топ-5 страниц, которые чаще всего посещают мужчины и топ-5 страниц, которые посещают чаще женщины.

from SlavicNames.parse_fio import gender_parse

def get_gender(name):
    return gender_parse(name).get('gender')

getGenderUDF = F.udf(lambda h: get_gender(h))

(
    df
    .join(df2.withColumn('gender', getGenderUDF(F.col('Full_name'))), df.id == df2.User_id, 'inner')
    .filter(F.col('gender') == 'm')
    .groupBy(df2.User_id)
    .agg(F.count(df2.User_id).alias("cnt"))
    .orderBy(F.col("cnt").desc())
    .show(5)    
)

# аналогично для женщин


+-------+---+
|User_id|cnt|
+-------+---+
|      1|  2|
+-------+---+



In [15]:
df.show(), df2.show()

+---+---------+------+-------+-----------------+-----+----+-----------+
| id|timestamp|  type|page_id|              tag| sign|hour|hour_window|
+---+---------+------+-------+-----------------+-----+----+-----------+
|  1|946683757| visit|      1|[политика, спорт]| true|   2|        0-4|
|  1|946687357| click|      2|          [спорт]| true|   3|        0-4|
|  3|946784557|scroll|      3|       [медицина]| true|   6|        4-8|
|  4|946975357|  move|      3|       [медицина]|false|  11|       8-12|
+---+---------+------+-------+-----------------+-----+----+-----------+

+---+-------+--------------------+-----------+
| Id|User_id|           Full_name|Create_date|
+---+-------+--------------------+-----------+
|  1|      1|Логинов Алексей А...| 1999-04-01|
|  2|      2|Калугин Максим Ль...| 1999-05-19|
|  3|      3|Кудрявцева Варвар...| 1999-09-05|
|  4|      4|Титова Василиса С...| 1999-12-01|
|  5|      5|Смирнова Екатерин...| 1999-01-01|
+---+-------+--------------------+-----------+


(None, None)